In [2]:
from selenium import webdriver
import re
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import pandas as pd

dr = webdriver.Chrome(ChromeDriverManager().install())

dr.get("https://www.screener.in/login/")

username = dr.find_element_by_id("id_username")
username.clear()
username.send_keys("skumarr53@gmail.com")

password = dr.find_element_by_id("id_password")
password.clear()
password.send_keys("Skumarr53@")

dr.find_element_by_css_selector('#main-area > form > p:nth-child(6) > button').click()


StockRatiosList = []

dr.get('https://www.screener.in/screen/raw/?sort=&order=&source=&query=Market+Capitalization%3E0')
lastpage = int(dr.find_element_by_css_selector('#main-area > div:nth-child(2) > div').text.split(' ')[-1])

for i  in range(1,lastpage+1): #lastpage
    url = 'https://www.screener.in/screen/raw/?sort=&source=&page='+str(i)+'&query=Market+Capitalization>0'
    dr.get(url)
    html = dr.page_source
    soup = bs(html)
    pageLinks = list(map(lambda x: x.get('href') ,soup.findAll('a',attrs={'href': re.compile("/company/")})))
    for j in pageLinks:
        stockUrl = 'https://www.screener.in'+j
        dr.get(stockUrl)
        txt = dr.find_element_by_css_selector('#main-area > section:nth-child(5)').text.replace(',','').replace(': ',':').replace(' %','').replace(' Cr.','').split('\n')
        dic = {i.split(':')[0]:i.split(':')[1] for i in txt if ':' in i}
        dic['Company'] = dr.find_element_by_css_selector('#company-info > h1').text
        SectorInd = dr.find_element_by_css_selector('#peers > h2 > small').text.replace('\n', "").replace('  ', "").split('//')
        dic['Sector'] = SectorInd[0]
        dic['Industry'] = SectorInd[1]
        
        StockRatiosList.append(dic)

    
data = pd.DataFrame(StockRatiosList)


Checking for win32 chromedriver:75.0.3770.90 in cache
Driver found in C:\Users\I20232\.wdm\chromedriver\75.0.3770.90\win32/chromedriver.exe


In [4]:
data.to_csv("StockFundamentalData.csv",index=False)

In [3]:
data.shape

(3791, 43)

In [38]:
data.tail()

,Asset Turnover Ratio,Average return on capital employed 3Years,Book Value,Book value,CROIC,Cash Conversion Cycle,Change in promoter holding 3Years,Company,Current Price,Current ratio,...,ROE,Receivables to Sales Ratio,Return on assets,Return on assets 3years,Sales Growth (3Yrs),Sector,Stock P/E,Unpledged promoter holding,Volume,debt to Free cash flow
3763,,%,23.16,NaN,22.77,,%,BMW Industries Ltd,25.30,1.39,...,%,,999,%,%,Sector: Steel,13.90,%,276,
3764,3.94,%,15.52,NaN,20.20,,%,Chandni Machines Ltd,6.04,1.30,...,%,0.20,999,999,%,Sector: Trading,2.38,46.50,33,
3765,3.13,%,8.32,NaN,6.29,,%,Anand Rayons Ltd,27.00,2.97,...,%,0.44,999,999,%,Sector: Trading,56.19,%,201000,
3766,3.17,%,55.57,NaN,-8.04,,%,Indiamart Intermesh Ltd,1347,0.47,...,%,0.00,6.43,353,25.90,Sector: E-Commerce/App based Aggregator,193.48,%,521782,0.00
3767,NaN,NaN,54.06,NaN,NaN,NaN,NaN,Mahindra EPC Irrigation Ltd,108.95,NaN,...,%,NaN,NaN,NaN,%,Sector: Plastic products,26.99,NaN,NaN,NaN
